In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from functions.words_chars import vocabulary_from_json_corpus

Using Theano backend.


In [7]:

json_corpus_path = "/home/ubuntu/summarization_query_oriented/data/wikipedia/json/td_qfs_rank_1/"
# building vocabulary of the corpus
words = vocabulary_from_json_corpus(json_corpus_path)
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))
print("word_indices", type(word_indices), "length:",len(word_indices))
print("indices_words", type(indices_word), "length", len(indices_word))

('word_indices', <type 'dict'>, 'length:', 83161)
('indices_words', <type 'dict'>, 'length', 83161)


In [8]:
maxlen = 10

In [9]:
#defining model
print('Build model...')
model = Sequential()
model.add(LSTM(400, return_sequences=True, input_shape=(maxlen, len(word_indices))))
model.add(Dropout(0.6))
model.add(LSTM(400, return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(len(words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print('Model built...')

Build model...
Model built...


In [10]:
model.load_weights('/home/ubuntu/summarization_query_oriented/nn_models/language_models/RNN/wider_30102016/tdqfs_lstm_wider_corpus_last.hdf5')

In [11]:
import gensim
import numpy as np

sentence_str = "Although they have a common name, diabetes mellitus and diabetes insipidus are two entirely separate conditions with unrelated mechanisms. "
sentence = gensim.utils.simple_preprocess(sentence_str)
starting_idx = 0
sentence = sentence[starting_idx :maxlen + starting_idx]

#start_index = random.randint(0, len(list_words) - maxlen - 1)

print()
print('----- Example:')
generated = ''
generated += ' '.join(sentence)
print('----- Generating with seed: "' , sentence , '"')
print()

for i in range(15):
    x = np.zeros((1, maxlen, len(words)))
    for t, word in enumerate(sentence):
        x[0, t, word_indices[word]] = 1.

    preds = model.predict(x, verbose=0)[0]

    next_index = np.argmax(preds)
    next_word = indices_word[next_index]
    generated += " " + next_word
    print generated
    del sentence[0]
    sentence.append(next_word)


()
----- Example:
('----- Generating with seed: "', [u'although', u'they', u'have', u'common', u'name', u'diabetes', u'mellitus', u'and', u'diabetes', u'insipidus'], '"')
()
although they have common name diabetes mellitus and diabetes insipidus the
although they have common name diabetes mellitus and diabetes insipidus the most
although they have common name diabetes mellitus and diabetes insipidus the most common
although they have common name diabetes mellitus and diabetes insipidus the most common symptoms
although they have common name diabetes mellitus and diabetes insipidus the most common symptoms of
although they have common name diabetes mellitus and diabetes insipidus the most common symptoms of the
although they have common name diabetes mellitus and diabetes insipidus the most common symptoms of the brain
although they have common name diabetes mellitus and diabetes insipidus the most common symptoms of the brain and
although they have common name diabetes mellitus and dia

# Comparison